preprocess.py (전체 흐름)

load & reorient & HU

resample 3D

skull strip & optional center crop

window & normalize (loop over experiments)

pad_or_crop_safe to shape.volume

for each axis, method → generate projection, apply same window/normalize, pad2d to shape.slice

torch.save with keys

In [ ]:
%cd C:\Users\401-09\Preprocessing

In [6]:
%run scripts/preprocess.py


: 

In [ ]:
# =========================================
# 저장된 .pt 파일 불러와 수치 검증
# =========================================

from pathlib import Path
import torch
import numpy as np

# case_id와 processed 폴더 경로
case_id = "049"
processed_dir = Path("data/processed")     # preprocess.py 에서 OUT_DIR 지정한 곳
pt_path = processed_dir / f"{case_id}.pt"

# 1) .pt 로드
data = torch.load(pt_path)

# 2) 이미지·마스크 꺼내기 (3D: [1, D, H, W] → [D, H, W])
image = data["volume"][0].numpy()    # scripts에서는 "volume" key 로 저장했으면 이걸로
mask  = data["mask"][0].numpy()

# 3) 기본 통계 출력
print("\n[Image Stats]")
print(f"  Mean:  {image.mean():.4f}")
print(f"  Std:   {image.std():.4f}")
print(f"  Min/Max: {image.min():.2f} / {image.max():.2f}")
print(f"  Shape: {image.shape}")
print(f"  Spacing: {data['meta']['spacing']}")

# 4) SNR / CNR 계산 (병변 픽셀 vs 배경 픽셀)
lesion     = image[mask == 1]
background = image[(mask == 0) & (image > 0)]  # skull strip 후 배경(0) 제외

if lesion.size > 0 and background.size > 0:
    snr = lesion.mean() / (lesion.std() + 1e-5)
    cnr = abs(lesion.mean() - background.mean()) / (background.std() + 1e-5)
    print("\n[Signal Quality]")
    print(f"  SNR: {snr:.4f}")
    print(f"  CNR: {cnr:.4f}")
else:
    print("\n[Signal Quality] 병변 또는 배경 데이터가 충분치 않음")
